In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError


In [20]:
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert text to lowercase and tokenize words
    words = text.lower().split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join words back into a single string
    cleaned_text = ' '.join(words)
    
    return cleaned_text



In [21]:
# Load datasets
fake_politifact = pd.read_csv('politifact_fake.csv')
fake_gossipcop = pd.read_csv('gossipcop_fake.csv')
abcnews = pd.read_csv('abcnews-date-text.csv')

# Assign labels: 0 for fake news, 1 for real news
fake_politifact['label'] = 0
fake_gossipcop['label'] = 0
abcnews['label'] = 1

# Preprocess the dataset
abcnews = abcnews.head(50000)  # Take the first 50,000 headlines
abcnews.rename(columns={'headline_text': 'title'}, inplace=True)  # Rename the column to match other datasets
abcnews['cleaned_title'] = abcnews['title'].apply(clean_text)  # Clean the headlines

# Concatenate datasets
data = pd.concat([fake_politifact, abcnews, fake_gossipcop], axis=0, ignore_index=True)
data['cleaned_title'] = data['title'].apply(clean_text)

In [22]:
# Split data into training and testing sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [23]:
train_data.shape

(44604, 7)

In [24]:
train_data.head()


,id,news_url,title,tweet_ids,label,publish_date,cleaned_title
33488,NaN,NaN,coria beats massu to second successive title,NaN,1,20030728.0,coria beats massu second successive title
50577,gossipcop-4394027963,www.intouchweekly.com/posts/kourtney-kardashia...,Pregnant Kourtney Kardashian Hangs out With Ju...,821041155534098435\t837002299952283649\t837002...,0,NaN,pregnant kourtney kardashian hangs justin bieb...
40281,NaN,NaN,man injured in two car collision on bass highway,NaN,1,20030830.0,man injured two car collision bass highway
24254,NaN,NaN,knights have last laugh in thriller,NaN,1,20030613.0,knights last laugh thriller
34623,NaN,NaN,aussies set the pace for team pursuit final,NaN,1,20030802.0,aussies set pace team pursuit final


In [25]:
test_data.shape

(11151, 7)

In [26]:
test_data.head()

,id,news_url,title,tweet_ids,label,publish_date,cleaned_title
37535,NaN,NaN,diabetes on the rise but death rate drops,NaN,1,20030816.0,diabetes rise death rate drops
3737,NaN,NaN,iraqi army has professional rapists vanstone,NaN,1,20030306.0,iraqi army professional rapists vanstone
55678,gossipcop-5035091613,NaN,Jennifer Aniston Brad Pitt Friends NOT Urg...,971088975438974977\t971093257127002112\t971130...,0,NaN,jennifer aniston brad pitt friends urging fair...
45581,NaN,NaN,roosters bulldogs clash sold out,NaN,1,20030926.0,roosters bulldogs clash sold
31592,NaN,NaN,phelps plays down thorpe rivalry,NaN,1,20030718.0,phelps plays thorpe rivalry


In [27]:
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

glove_vectors = load_glove_vectors('glove.6B.50d.txt')


In [28]:
def weighted_average_vectors(docs, vectorizer, word_vectors, dim=50):
    # Calculate the tf-idf weights for the given documents
    tfidf_matrix = vectorizer.fit_transform(docs)

    # Initialize an empty matrix to store the weighted average vectors
    weighted_vectors = np.zeros((len(docs), dim))

    # Iterate through the documents and compute the weighted average vector for each
    for i, doc in enumerate(docs):
        words = doc.split()
        weighted_sum = np.zeros(dim)
        total_weight = 0

        for word in words:
            if word in word_vectors and word in vectorizer.vocabulary_:
                vector = word_vectors[word]
                weight = tfidf_matrix[i, vectorizer.vocabulary_[word]]
                weighted_sum += weight * vector
                total_weight += weight

        if total_weight != 0:
            weighted_vectors[i] = weighted_sum / total_weight

    return weighted_vectors


In [29]:
vectorizer = TfidfVectorizer(max_features=10000)
X_train = weighted_average_vectors(train_data['cleaned_title'], vectorizer, glove_vectors)
X_test = weighted_average_vectors(test_data['cleaned_title'], vectorizer, glove_vectors)

y_train = train_data['label'].values
y_test = test_data['label'].values


In [30]:
def create_autoencoder(input_dim, encoding_dim, hidden_layers):
    # Encoder
    encoder_input = Input(shape=(input_dim,))
    encoder_layers = [Dense(hidden_layers[0], activation='relu')(encoder_input)]

    for layer_size in hidden_layers[1:]:
        encoder_layers.append(Dense(layer_size, activation='relu')(encoder_layers[-1]))

    # Encoded representation
    encoded = Dense(encoding_dim, activation='relu')(encoder_layers[-1])

    # Decoder
    decoder_layers = [Dense(hidden_layers[-1], activation='relu')(encoded)]

    for layer_size in reversed(hidden_layers[:-1]):
        decoder_layers.append(Dense(layer_size, activation='relu')(decoder_layers[-1]))

    # Reconstruction
    decoder_output = Dense(input_dim, activation='linear')(decoder_layers[-1])

    # Build the autoencoder model
    autoencoder = Model(encoder_input, decoder_output)
    return autoencoder


In [31]:
input_dim = X_train.shape[1]
encoding_dim = 32
hidden_layers = [128, 64]

autoencoder = create_autoencoder(input_dim, encoding_dim, hidden_layers)
autoencoder.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50)]              0         
                                                                 
 dense_6 (Dense)             (None, 128)               6528      
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 64)                2112      
                                                                 
 dense_10 (Dense)            (None, 128)               8320      
                                                                 
 dense_11 (Dense)            (None, 50)                6450

In [32]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
loss = MeanSquaredError()
autoencoder.compile(optimizer=optimizer, loss=loss)

# Train the model
real_headlines = X_train[y_train == 1]  # filter real news headlines
epochs = 100
batch_size = 32
autoencoder.fit(real_headlines, real_headlines, epochs=epochs, batch_size=batch_size, validation_split=0.1)


Epoch 1/100
1126/1126 [==============================] - 2s 1ms/step - loss: 0.0447 - val_loss: 0.0287
Epoch 2/100
1126/1126 [==============================] - 2s 1ms/step - loss: 0.0278 - val_loss: 0.0269
Epoch 3/100
1126/1126 [==============================] - 1s 1ms/step - loss: 0.0266 - val_loss: 0.0261
Epoch 4/100
1126/1126 [==============================] - 1s 1ms/step - loss: 0.0246 - val_loss: 0.0240
Epoch 5/100
1126/1126 [==============================] - 1s 1ms/step - loss: 0.0237 - val_loss: 0.0236
Epoch 6/100
1126/1126 [==============================] - 1s 1ms/step - loss: 0.0231 - val_loss: 0.0228
Epoch 7/100
1126/1126 [==============================] - 1s 1ms/step - loss: 0.0227 - val_loss: 0.0226
Epoch 8/100
1126/1126 [==============================] - 1s 1ms/step - loss: 0.0224 - val_loss: 0.0226
Epoch 9/100
1126/1126 [==============================] - 2s 1ms/step - loss: 0.0221 - val_loss: 0.0229
Epoch 10/100
1126/1126 [==============================] - 1s 1ms/step - l

In [33]:
# Reconstruct headlines in the test set
X_test_reconstructed = autoencoder.predict(X_test)

# Calculate the reconstruction errors
reconstruction_errors = np.mean((X_test - X_test_reconstructed)**2, axis=1)


349/349 [==============================] - 0s 781us/step


In [34]:
from sklearn.metrics import confusion_matrix

def find_optimal_threshold(errors, y_true):
    min_diff = float('inf')
    optimal_threshold = 0

    for threshold in np.linspace(errors.min(), errors.max(), num=1000):
        y_pred = (errors > threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        fpr = fp / (fp + tn)  # False Positive Rate
        fnr = fn / (fn + tp)  # False Negative Rate

        diff = abs(fpr - fnr)

        if diff < min_diff:
            min_diff = diff
            optimal_threshold = threshold

    return optimal_threshold

threshold = find_optimal_threshold(reconstruction_errors, y_test)


In [40]:
y_pred = (reconstruction_errors > threshold).astype(int)


In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Autoencoder Classifier Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")



Autoencoder Classifier Metrics:
Accuracy: 0.43063402385436284
Precision: 0.8670169765561844
Recall: 0.4298166516381124
F1 Score: 0.5747203429566616


In [42]:
from sklearn.linear_model import LogisticRegression

# Train the logistic regression model with an increased number of iterations
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Make predictions
y_pred_log_reg = log_reg.predict(X_test)

# Calculate metrics
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
precision_log_reg = precision_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg)

print(f"Logistic Regression Metrics:\nAccuracy: {accuracy_log_reg}\nPrecision: {precision_log_reg}\nRecall: {recall_log_reg}\nF1 Score: {f1_log_reg}")


Logistic Regression Metrics:
Accuracy: 0.9598242310106717
Precision: 0.9699300009859017
Recall: 0.9856727782787296
F1 Score: 0.9777380242496521


In [44]:
from sklearn.ensemble import RandomForestClassifier

# Train the random forest model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

# Calculate metrics
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest Metrics:\nAccuracy: {accuracy_rf}\nPrecision: {precision_rf}\nRecall: {recall_rf}\nF1 Score: {f1_rf}")


Random Forest Metrics:
Accuracy: 0.9639494215765402
Precision: 0.9652258377853327
Recall: 0.995591624085763
F1 Score: 0.9801736042611955
